# <font color="#418FDE" size="6.5" uppercase>**Tensors and Ops**</font>

>Last update: 20260125.
    
By the end of this Lecture, you will be able to:
- Create and manipulate TensorFlow tensors with specified shapes and dtypes. 
- Apply common TensorFlow math and array operations to transform tensors. 
- Explain TensorFlow broadcasting and shape inference in simple expressions. 


## **1. Creating Tensors**

### **1.1. Constants and Variables**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Master TensorFlow 2.20.0/Module_02/Lecture_A/image_01_01.jpg?v=1769366658" width="250">



>* Constant tensors hold fixed values during computation
>* Variable tensors store updatable values for learning

>* Define constants with values, shape, and dtype
>* Used in computations while their values stay fixed

>* Variable tensors change values but keep shape
>* They start from initial values and update during training



In [ ]:
#@title Python Code - Constants and Variables

# This script introduces TensorFlow constants and variables.
# It focuses on shapes and data types for tensors.
# Run each section and observe the printed outputs.

# Install TensorFlow if not already available in the environment.
# !pip install tensorflow==2.20.0 --quiet.

# Import TensorFlow with a short alias.
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
import tensorflow as tf

# Print TensorFlow version in one concise line.
print("TensorFlow version:", tf.__version__)

# Create a scalar constant with explicit float32 dtype.
scalar_const = tf.constant(3.5, dtype=tf.float32)

# Create a 1D constant tensor from a Python list.
vector_const = tf.constant([1, 2, 3], dtype=tf.int32)

# Create a 2D constant tensor representing a small matrix.
matrix_const = tf.constant([[1., 2.], [3., 4.]], dtype=tf.float32)

# Print basic information about the scalar constant.
print("Scalar constant:", scalar_const.numpy(), scalar_const.shape)

# Print basic information about the vector constant.
print("Vector constant:", vector_const.numpy(), vector_const.shape)

# Print basic information about the matrix constant.
print("Matrix constant:\n", matrix_const.numpy())

# Demonstrate that constants cannot be reassigned in place.
new_matrix_const = matrix_const + tf.constant(1.0, dtype=tf.float32)

# Show original and new constant values for comparison.
print("Original matrix constant:\n", matrix_const.numpy())

# Show the result of adding one to the matrix constant.
print("New matrix constant plus one:\n", new_matrix_const.numpy())

# Create a variable tensor from an initial constant value.
var_weights = tf.Variable([[0.5, -0.5], [1.0, -1.0]], dtype=tf.float32)

# Print the initial variable value and its shape.
print("Initial variable weights:\n", var_weights.numpy())

# Define a small update tensor with the same shape as the variable.
update_step = tf.constant([[0.1, 0.1], [0.1, 0.1]], dtype=tf.float32)

# Validate that shapes match before applying the update.
if var_weights.shape == update_step.shape:
    # Apply an in place style update using assign_add.
    var_weights.assign_add(update_step)

# Print the updated variable values after the assign_add.
print("Updated variable weights:\n", var_weights.numpy())

# Create another variable to show dtype must stay consistent.
var_bias = tf.Variable([0.0, 0.0], dtype=tf.float32)

# Safely update the bias variable with a matching dtype tensor.
var_bias.assign(var_bias + tf.constant([0.2, -0.1], dtype=tf.float32))

# Print final bias values to confirm the successful update.
print("Updated bias variable:", var_bias.numpy())



### **1.2. Random and Zero Tensors**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Master TensorFlow 2.20.0/Module_02/Lecture_A/image_01_02.jpg?v=1769366730" width="250">



>* Use random or zero tensors for initialization
>* Choose tensor shape and dtype for compatibility

>* Choose random distributions to match modeling needs
>* Set shape and dtype; framework fills values

>* Zero or constant tensors give neutral placeholders
>* Match shape and dtype to avoid errors



In [ ]:
#@title Python Code - Random and Zero Tensors

# This script explores random and zero tensors.
# It uses TensorFlow to create simple tensors.
# Focus on shapes dtypes and deterministic randomness.

# !pip install tensorflow==2.20.0.

# Import TensorFlow with a clear alias.
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
import tensorflow as tf

# Print TensorFlow version briefly.
print("TensorFlow version:", tf.__version__)

# Set global random seed for reproducibility.
tf.random.set_seed(42)

# Create a small random tensor with normal distribution.
random_normal = tf.random.normal(
    shape=(2, 3), mean=0.0, stddev=1.0, dtype=tf.float32
)

# Create a small random tensor with uniform distribution.
random_uniform = tf.random.uniform(
    shape=(2, 3), minval=0.0, maxval=1.0, dtype=tf.float32
)

# Create a zero tensor matching a specific shape.
zeros_explicit = tf.zeros(shape=(2, 3), dtype=tf.float32)

# Create a zero tensor that matches another tensor shape.
zeros_like_random = tf.zeros_like(random_normal, dtype=tf.float32)

# Show shapes and dtypes for all created tensors.
print("random_normal shape and dtype:", random_normal.shape, random_normal.dtype)
print("random_uniform shape and dtype:", random_uniform.shape, random_uniform.dtype)
print("zeros_explicit shape and dtype:", zeros_explicit.shape, zeros_explicit.dtype)
print("zeros_like_random shape and dtype:", zeros_like_random.shape, zeros_like_random.dtype)

# Safely check that shapes match before an addition.
if random_normal.shape == zeros_like_random.shape:
    added_tensor = random_normal + zeros_like_random
else:
    added_tensor = None

# Print a small sample of tensor values.
print("random_normal values:\n", random_normal.numpy())
print("zeros_like_random values:\n", zeros_like_random.numpy())

# Confirm that adding zeros keeps values unchanged.
print("added_tensor equals random_normal:", tf.reduce_all(
    tf.equal(added_tensor, random_normal)
).numpy())



### **1.3. From NumPy to Tensors**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Master TensorFlow 2.20.0/Module_02/Lecture_A/image_01_03.jpg?v=1769366795" width="250">



>* Array libraries handle rich, real-world data structures
>* Converting arrays to tensors enables optimized ML computation

>* Tensor keeps array values, shape, and structure
>* Choose tensor dtypes carefully to avoid subtle bugs

>* Converting arrays to tensors connects raw data
>* Tensors enable efficient computation while preserving structure



In [ ]:
#@title Python Code - From NumPy to Tensors

# This script shows NumPy to TensorFlow tensor conversion.
# It focuses on shapes and data types for tensors.
# Run each part and observe the printed outputs.

# !pip install tensorflow.
# !pip install numpy.

# Import required standard libraries safely.
import os
import sys
import math

# Try importing numpy and handle missing installation.
try:
    import numpy as np
except ImportError as exc:
    raise SystemExit("NumPy is required for this script.") from exc

# Try importing tensorflow and handle missing installation.
try:
    import tensorflow as tf
except ImportError as exc:
    raise SystemExit("TensorFlow is required for this script.") from exc

# Print TensorFlow version in one short line.
print("TensorFlow version:", tf.__version__)

# Create a small NumPy array with float values.
numpy_array = np.array([[1.0, 2.0], [3.0, 4.0]], dtype=np.float32)

# Show basic information about the NumPy array.
print("NumPy array shape:", numpy_array.shape)
print("NumPy array dtype:", numpy_array.dtype)

# Convert NumPy array to a TensorFlow tensor.
tensor_default = tf.convert_to_tensor(numpy_array)

# Show tensor shape and dtype after conversion.
print("Tensor shape (default):", tensor_default.shape)
print("Tensor dtype (default):", tensor_default.dtype)

# Convert NumPy array to tensor with explicit float64 dtype.
tensor_float64 = tf.convert_to_tensor(numpy_array, dtype=tf.float64)

# Print dtype change while shape stays the same.
print("Tensor shape (float64):", tensor_float64.shape)
print("Tensor dtype (float64):", tensor_float64.dtype)

# Create integer NumPy array to show integer tensor conversion.
int_numpy_array = np.array([[0, 1, 2], [3, 4, 5]], dtype=np.int32)

# Convert integer NumPy array to integer tensor explicitly.
int_tensor = tf.convert_to_tensor(int_numpy_array, dtype=tf.int32)

# Print integer tensor properties in a compact way.
print("Int tensor shape:", int_tensor.shape)
print("Int tensor dtype:", int_tensor.dtype)

# Validate that tensor and NumPy shapes are identical.
if tensor_default.shape != numpy_array.shape:
    raise ValueError("Tensor and NumPy shapes do not match.")

# Show that small tensor values match original NumPy values.
print("First row NumPy:", numpy_array[0])
print("First row tensor:", tensor_default.numpy()[0])




## **2. Core Tensor Operations**

### **2.1. Elementwise Tensor Operations**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Master TensorFlow 2.20.0/Module_02/Lecture_A/image_02_01.jpg?v=1769366840" width="250">



>* Same operation applied independently to all elements
>* Efficient, spreadsheet-like transforms crucial for computation

>* Elementwise ops apply many math functions per entry
>* They reshape data scale while preserving structure

>* Combine data with parameters using elementwise transforms
>* Local per-element updates build complex global behavior



In [ ]:
#@title Python Code - Elementwise Tensor Operations

# This script demonstrates elementwise tensor operations.
# It uses only built in Python and lists.
# Focus is on shapes broadcasting and transformations.

# Create two small one dimensional lists as tensors.
a = [1.0, 2.0, 3.0]
# Create another list with same length for pairing.
b = [10.0, 20.0, 30.0]
# Show the original lists for reference.
print("a:", a)

# Show the second list for comparison.
print("b:", b)
# Perform elementwise addition using list comprehension.
add_result = [x + y for x, y in zip(a, b)]
# Perform elementwise multiplication similarly.
mul_result = [x * y for x, y in zip(a, b)]
# Print the elementwise addition result.
print("a + b:", add_result)

# Print the elementwise multiplication result.
print("a * b:", mul_result)
# Define a scalar to apply elementwise to list.
scalar = 0.5
# Scale each element of list a by scalar.
scaled_a = [scalar * x for x in a]
# Print the scaled list to show transformation.
print("0.5 * a:", scaled_a)

# Apply an elementwise nonlinear transformation.
squared_a = [x ** 2 for x in a]
# Print squared values to show nonlinear effect.
print("a squared:", squared_a)
# Clip values elementwise between given bounds.
clipped_a = [min(max(x, 1.5), 2.5) for x in a]
# Print clipped values to show independent changes.
print("clipped a:", clipped_a)

# Demonstrate simple broadcasting with different length lists.
short = [1.0, 2.0]
# Validate compatible lengths for manual broadcasting.
if len(a) % len(short) != 0:
    pass
# Repeat short list pattern to match length of a.
short_tiled = (short * (len(a) // len(short)))[: len(a)]

# Perform elementwise multiplication with tiled pattern.
broadcast_mul = [x * y for x, y in zip(a, short_tiled)]
# Print pattern and result to explain broadcasting.
print("short tiled:", short_tiled)
print("a * short_tiled:", broadcast_mul)



### **2.2. Matrix Multiplication Basics**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Master TensorFlow 2.20.0/Module_02/Lecture_A/image_02_02.jpg?v=1769366886" width="250">



>* Matrix multiplication mixes data with learned parameters
>* Used to reshape features and propagate network activations

>* Matrix multiplication pairs rows with matching columns
>* TensorFlow checks inner dimensions and hardware efficiency

>* Use batched matrix multiplies for many examples
>* Leverage batch dimensions to exploit optimized routines



In [ ]:
#@title Python Code - Matrix Multiplication Basics

# This script demonstrates basic matrix multiplication.
# It uses tiny tensors to keep shapes understandable.
# Focus on shapes and results not heavy computation.

# import TensorFlow for tensor operations.
import tensorflow as tf

# print TensorFlow version for reproducibility.
print("TensorFlow version:", tf.__version__)

# create a simple 2x3 matrix of features.
features = tf.constant([[1.0, 2.0, 3.0],
                        [4.0, 5.0, 6.0]])

# create a 3x2 weight matrix for transformation.
weights = tf.constant([[0.1, 0.2],
                       [0.3, 0.4],
                       [0.5, 0.6]])

# show shapes so multiplication rule is clear.
print("features shape:", features.shape)
print("weights shape:", weights.shape)

# validate inner dimensions match for matmul.
if features.shape[1] != weights.shape[0]:
    raise ValueError("Inner dimensions must match for matmul")

# perform matrix multiplication using tf.matmul.
outputs = tf.matmul(features, weights)

# show resulting shape after multiplication.
print("outputs shape:", outputs.shape)

# print small matrices to see numeric effect.
print("features matrix:\n", features.numpy())
print("weights matrix:\n", weights.numpy())
print("outputs matrix:\n", outputs.numpy())

# demonstrate batched matrix multiplication basics.
batch_features = tf.stack([features, features], axis=0)

# confirm new batched tensor shape.
print("batch_features shape:", batch_features.shape)

# apply same weights to each batch element.
batch_outputs = tf.matmul(batch_features, weights)

# print final batched outputs shape only.
print("batch_outputs shape:", batch_outputs.shape)



### **2.3. Tensor Reduction Operations**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Master TensorFlow 2.20.0/Module_02/Lecture_A/image_02_03.jpg?v=1769366930" width="250">



>* Reductions summarize many tensor values into fewer
>* They compute sums, averages, extremes for model use

>* Choosing reduction axes collapses specific tensor dimensions
>* Axis choice controls output shape and interpretation

>* Use specialized reductions to capture key statistics
>* Chain reductions to normalize, aggregate, and guide learning



In [ ]:
#@title Python Code - Tensor Reduction Operations

# This script demonstrates TensorFlow tensor reductions.
# It focuses on simple shapes and clear outputs.
# Run each part and read printed explanations.

# Install TensorFlow if not already available.
# !pip install tensorflow-cpu==2.20.0.

# Import TensorFlow with a short alias.
import tensorflow as tf

# Disable GPU to avoid CUDA_ERROR_INVALID_HANDLE errors on some systems.
try:
    tf.config.set_visible_devices([], 'GPU')
except Exception:
    pass

# Print TensorFlow version for reproducibility.
print("TensorFlow version:", tf.__version__)

# Create a small 2D tensor of floats.
values_2d = tf.constant([[1.0, 2.0, 3.0],
                         [4.0, 5.0, 6.0]])

# Show the tensor and its shape briefly.
print("values_2d:", values_2d.numpy())

# Compute the global sum over all elements.
global_sum = tf.reduce_sum(values_2d)

# Print the scalar global sum result.
print("Global sum:", global_sum.numpy())

# Reduce along axis zero keeping column wise sums.
col_sum = tf.reduce_sum(values_2d, axis=0)

# Print column wise sums and resulting shape.
print("Column sum, shape", col_sum.shape, ":", col_sum.numpy())

# Reduce along axis one keeping row wise sums.
row_sum = tf.reduce_sum(values_2d, axis=1)

# Print row wise sums and resulting shape.
print("Row sum, shape", row_sum.shape, ":", row_sum.numpy())

# Compute mean over all elements as scalar.
global_mean = tf.reduce_mean(values_2d)

# Print the scalar mean value clearly.
print("Global mean:", global_mean.numpy())

# Compute maximum along axis zero for each column.
col_max = tf.reduce_max(values_2d, axis=0)

# Print column maximum values and their shape.
print("Column max, shape", col_max.shape, ":", col_max.numpy())

# Create a boolean tensor for nonzero counting.
nonzero_mask = tf.not_equal(values_2d, 0.0)

# Count nonzero elements using reduce_sum on mask.
nonzero_count = tf.reduce_sum(tf.cast(nonzero_mask, tf.int32))

# Final print summarizing nonzero element count.
print("Nonzero element count:", int(nonzero_count.numpy()))



## **3. Tensor Shapes and Broadcasting**

### **3.1. Static and Dynamic Shapes**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Master TensorFlow 2.20.0/Module_02/Lecture_A/image_03_01.jpg?v=1769366998" width="250">



>* Static shape is TensorFlow’s known tensor layout
>* It catches shape errors early and optimizes computation

>* Some tensor dimensions stay unknown until runtime
>* TensorFlow uses partial shapes, final sizes resolve later

>* Static shapes enable early, predictable broadcasting checks
>* Dynamic shapes defer compatibility errors until runtime



In [ ]:
#@title Python Code - Static and Dynamic Shapes

# This script explores static and dynamic shapes.
# It uses TensorFlow tensors to show shape behavior.
# Focus on broadcasting and simple shape inference.

# Uncomment the next line if TensorFlow is missing.
# !pip install tensorflow==2.20.0.

# Import TensorFlow with a clear alias.
import tensorflow as tf

# Force TensorFlow to use CPU only to avoid CUDA errors.
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

# Print TensorFlow version for reproducibility.
print("TensorFlow version:", tf.__version__)

# Create a simple constant tensor with known shape.
static_tensor = tf.constant([[1.0, 2.0], [3.0, 4.0]])

# Show the static shape known at definition time.
print("Static shape of tensor:", static_tensor.shape)

# Show the dynamic shape evaluated at runtime.
print("Dynamic shape via tf.shape:", tf.shape(static_tensor))

# Create a placeholder like tensor using tf.zeros.
batch_size = 3

# Define a tensor where batch dimension can change.
flexible_tensor = tf.zeros((batch_size, 4))

# Show static shape information for flexible tensor.
print("Static shape flexible:", flexible_tensor.shape)

# Show dynamic shape information for flexible tensor.
print("Dynamic shape flexible:", tf.shape(flexible_tensor))

# Simulate unknown batch by using None in shape.
example_spec = tf.TensorSpec(shape=(None, 4), dtype=tf.float32)

# Print the static shape from the TensorSpec.
print("TensorSpec static shape:", example_spec.shape)

# Create a new batch with different batch size.
new_batch = tf.ones((5, 4), dtype=tf.float32)

# Confirm that new batch matches TensorSpec rank.
print("New batch static shape:", new_batch.shape)

# Use tf.shape to get dynamic shape of new batch.
print("New batch dynamic shape:", tf.shape(new_batch))

# Create a bias vector that will broadcast.
bias = tf.constant([0.5, 0.5, 0.5, 0.5], dtype=tf.float32)

# Show shapes before broadcasting addition.
print("Bias static shape:", bias.shape)

# Perform broadcasting addition with new_batch and bias.
result = new_batch + bias

# Show static shape of the broadcasted result.
print("Result static shape:", result.shape)

# Show dynamic shape of the broadcasted result.
print("Result dynamic shape:", tf.shape(result))



### **3.2. Broadcasting Rules**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Master TensorFlow 2.20.0/Module_02/Lecture_A/image_03_02.jpg?v=1769367059" width="250">



>* Broadcasting aligns tensor shapes right to left
>* Size-one dimensions stretch to match larger tensors

>* Image batch and channel corrections have compatible shapes
>* Broadcasting repeats three corrections across every image pixel

>* Incompatible dimensions cause broadcasting shape mismatch errors
>* Understand which axes stretch to avoid wrong results



In [ ]:
#@title Python Code - Broadcasting Rules

# This script demonstrates TensorFlow broadcasting rules.
# It focuses on simple shapes and clear outputs.
# Run each part and read printed explanations.

# Install TensorFlow if not already available.
# pip install tensorflow==2.20.0.

# Import TensorFlow with a short alias.
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
import tensorflow as tf

# Print TensorFlow version for reproducibility.
print("TensorFlow version:", tf.__version__)

# Create a base tensor representing a week of sales.
week_sales = tf.constant([10., 12., 9., 11., 13., 8., 7.])

# Show the shape of the base tensor.
print("week_sales shape:", week_sales.shape)

# Create a scalar bias that will be broadcast.
scalar_bias = tf.constant(1.5)

# Add scalar to vector using broadcasting.
sales_plus_scalar = week_sales + scalar_bias

# Print result and shape after scalar broadcasting.
print("scalar broadcast result:", sales_plus_scalar.numpy())

# Create a per day adjustment vector with matching shape.
per_day_adjust = tf.constant([1., 0., -1., 0., 1., 0., -1.])

# Add per day adjustments without broadcasting changes.
sales_plus_day = week_sales + per_day_adjust

# Print result and confirm same shape as original.
print("per day result shape:", sales_plus_day.shape)

# Create a matrix representing two weeks of sales data.
two_weeks = tf.stack([week_sales, week_sales + 2.])

# Print the shape of the two weeks tensor.
print("two_weeks shape:", two_weeks.shape)

# Reshape per day adjustment for column broadcasting.
per_day_column = tf.reshape(per_day_adjust, (1, 7))

# Add row vector to each week using broadcasting.
weeks_plus_day = two_weeks + per_day_column

# Print shape after row wise broadcasting.
print("weeks_plus_day shape:", weeks_plus_day.shape)

# Create a column bias that broadcasts over days.
per_week_bias = tf.reshape(tf.constant([2., -1.]), (2, 1))

# Add column bias to two weeks tensor using broadcasting.
weeks_plus_bias = two_weeks + per_week_bias

# Print shape after column wise broadcasting.
print("weeks_plus_bias shape:", weeks_plus_bias.shape)

# Try an incompatible shape and handle the error.
try:
    bad_adjust = tf.constant([1., 2., 3., 4., 5.])
    _ = week_sales + bad_adjust
except Exception as e:
    print("incompatible broadcast error type:", type(e).__name__)



### **3.3. Reshaping Tensors Safely**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Master TensorFlow 2.20.0/Module_02/Lecture_A/image_03_03.jpg?v=1769367122" width="250">



>* Reshaping changes tensor dimensions, not the data
>* Total element count must stay exactly constant

>* Match reshaped tensors with later operation shapes
>* Keep each dimension’s real-world meaning consistent

>* Be cautious using inferred dimensions when reshaping
>* Verify axes meanings to avoid harmful broadcasting



In [ ]:
#@title Python Code - Reshaping Tensors Safely

# This script explores safe tensor reshaping.
# It uses only Python lists and reasoning.
# Focus on shapes, sizes, and safe changes.

# Show an original flat list representing data.
original_data = [1, 2, 3, 4, 5, 6]

# Explain the original length as total elements.
original_length = len(original_data)

# Print the original data and its length.
print("Original data:", original_data)
print("Original length:", original_length)

# Define a helper to compute product of dimensions.
def product(dims):
    result = 1
    for d in dims:
        result *= d
    return result

# Choose a target shape that is compatible.
target_shape_safe = (2, 3)

# Compute element count for the safe shape.
safe_count = product(target_shape_safe)

# Print the safe shape and its element count.
print("Safe shape:", target_shape_safe, "elements:", safe_count)

# Check if safe shape matches original length.
if safe_count == original_length:
    print("Safe reshape possible, sizes match.")
else:
    print("Safe reshape not possible here.")

# Choose an unsafe target shape on purpose.
target_shape_unsafe = (4, 3)

# Compute element count for the unsafe shape.
unsafe_count = product(target_shape_unsafe)

# Print the unsafe shape and its element count.
print("Unsafe shape:", target_shape_unsafe, "elements:", unsafe_count)

# Check and warn if unsafe reshape is attempted.
if unsafe_count != original_length:
    print("Warning: cannot reshape, element counts differ.")
else:
    print("Unexpectedly safe, counts are equal.")

# Demonstrate a simple manual reshape into rows.
rows, cols = target_shape_safe

# Build a nested list to mimic a reshaped tensor.
reshaped = [original_data[i * cols:(i + 1) * cols] for i in range(rows)]

# Print the reshaped structure and its dimensions.
print("Reshaped data:", reshaped, "with shape", target_shape_safe)



# <font color="#418FDE" size="6.5" uppercase>**Tensors and Ops**</font>


In this lecture, you learned to:
- Create and manipulate TensorFlow tensors with specified shapes and dtypes. 
- Apply common TensorFlow math and array operations to transform tensors. 
- Explain TensorFlow broadcasting and shape inference in simple expressions. 

In the next Lecture (Lecture B), we will go over 'Autograd with TF'